passo1) aqui são importadas todas as bibliotecas e definidos todos os modelos que serão utilizados, além disso, manualmente aqui são indicados os paths do video que deverá passar pelo processo definido no script. 
- instance_dir = diretório da instância... usado como espaço de armazenamento para os folders, arrays e csv's resultantes do processo
- instance_id = identificador unico da instância (aqui pode ser definido manualmente, mas é prudente que seja o mesmo nome da instância e que nomes de instancia não se repitam)
- group_size = indica qual o numero de frames a serem extraídas do video (aleatoriamente uma vez que esta sendo usado o comando "get_rando_frame_group")
- vidpath = path do video de onde os frames devem ser extraídos

In [1]:
import numpy as np
import os
from keras import applications
from tools import evectools as evt
import pandas as pd
from keras import applications


############################This section contains all information necessary previous to the execution of the script#################################################################
df = pd.DataFrame()
instance_dir = 'C:/Documents/Projects_IC/creativision_nyu/UrbanCreativision/evec_scan/instances/test2'
vid_file = 'video2.mp4'
instance_id = 'test2'
group_size = 6
vidpath = 'C:/Documents/Projects_IC/creativision_nyu/UrbanCreativision/evec_scan/instances/test2/video2.mp4' 
description_models = ['moondream', 'llava-llama3'] #'moondream', 'llava-llama3'
description_embedding_models = {"model":['mxbai-embed-large'], "model_origin":['ollama']} #{"model":['mxbai-embed-large', "ViT-B/32"], "model_origin":['ollama', "clip"]} #'mxbai-embed-large','nomic-embed-text', 'all-minilm' from ollama
image_embedding_models = {'model':[applications.VGG16, "ViT-B/32"], 'model_origin':['keras', 'clip']} #applications.VGG16, applications.DenseNet121


passo2) esta próxima seção utiliza ferramentas definidas em evectools.py para extrair o grupo de frames e gerar uma coluna numa dataframe com todas as frames extraídas. No processo, usa "save_group" para gerar um folder onde cada frame é salva com seu respectivo nome e numero de identificação. Os paths dessas frames são so valores salvos no dataframe.

In [2]:
############################This section organizes a small group of frames from the video being analyzed###########################################################################
video_instance = evt.FrameExtractor(vidpath=vidpath)
frame_group = video_instance.get_random_frame_group(size=group_size)
frame_group_instance = video_instance.FrameGroup(frame_extractor=video_instance, group_set=frame_group)
frames_output_path = os.path.join(instance_dir, 'frames')
frame_path_list = frame_group_instance.save_group(output_dir=frames_output_path, group_name=instance_id)
df['frame_path'] = frame_path_list
df.index = list(range(len(frame_path_list)))


file exists:  True
{'streams': [{'index': 0, 'codec_name': 'h264', 'codec_long_name': 'H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10', 'profile': 'High', 'codec_type': 'video', 'codec_tag_string': 'avc1', 'codec_tag': '0x31637661', 'width': 1920, 'height': 1080, 'coded_width': 1920, 'coded_height': 1080, 'closed_captions': 0, 'film_grain': 0, 'has_b_frames': 0, 'pix_fmt': 'yuvj420p', 'level': 40, 'color_range': 'pc', 'color_space': 'bt470bg', 'color_transfer': 'smpte170m', 'color_primaries': 'bt470bg', 'chroma_location': 'left', 'field_order': 'progressive', 'refs': 1, 'is_avc': 'true', 'nal_length_size': '4', 'id': '0x1', 'r_frame_rate': '30/1', 'avg_frame_rate': '1113840000/37118533', 'time_base': '1/90000', 'start_pts': 0, 'start_time': '0.000000', 'duration_ts': 74237066, 'duration': '824.856289', 'bit_rate': '20006092', 'bits_per_raw_sample': '8', 'nb_frames': '24752', 'extradata_size': 35, 'disposition': {'default': 1, 'dub': 0, 'original': 0, 'comment': 0, 'lyrics': 0, 'karaoke': 0,

passo3) aqui são geradas as descrições de cada frame. A lógica utilizada é: Em ordem, para cada frame no dataframe, um primeiro modelo irá gerar a descrição. Na sequência, ele irá salvar essa descrição em uma nova coluna no dataframe, tal que, devido a ordem com que as descrições são geradas, alinhará cada frame com sua respectiva descrição feita por esse modelo. Na sequência um próximo modelo irá gerar as descrições de cada frame e gerar uma nova coluna. O nome da coluna é o nome do modelo usado. No final o dataframe resultante será salvo em um csv

In [3]:
#############################This section gets the descriptions for every frame in the previous group model by model###########################################################################
description_columns = []
for desc_model in description_models: 
    descriptions_per_model = []
    for frame_path in df['frame_path']:
        descriptor = evt.ImgDescriptor(model=desc_model,frame_path=frame_path)
        description = descriptor.get_description()
        descriptions_per_model.append(description)
    df[f'{desc_model}_desc'] = descriptions_per_model
    description_columns.append(f'{desc_model}_desc')
df_output_path = os.path.join(instance_dir, 'instance_data.csv')
df.to_csv(df_output_path, index=True)



segue o dataframe resultante do passo anterior

In [4]:
instance_data = pd.read_csv('C:/Documents/Projects_IC/creativision_nyu/UrbanCreativision/evec_scan/instances/test1/instance_data.csv')
instance_data.drop('Unnamed: 0', axis=1)

,frame_path,moondream_desc,llava-llama3_desc
0,C:/Documents/Projects_IC/creativision_nyu/Urba...,\nThe image shows a bus stop on the side of a ...,The image captures a vibrant scene from a stre...
1,C:/Documents/Projects_IC/creativision_nyu/Urba...,\nThe image shows a bus stop on the side of a ...,The image captures a serene urban scene. Domin...
2,C:/Documents/Projects_IC/creativision_nyu/Urba...,\nThe image shows a bus stop on the side of a ...,The image captures a tranquil scene at the ent...
3,C:/Documents/Projects_IC/creativision_nyu/Urba...,\nThe image shows a bus stop on the side of a ...,The image captures a serene urban scene. Domin...
4,C:/Documents/Projects_IC/creativision_nyu/Urba...,\nThe image shows a bus stop on the side of a ...,The image captures a tranquil scene in an urba...
5,C:/Documents/Projects_IC/creativision_nyu/Urba...,\nThe image shows a bus stop on the side of a ...,The image captures a scene at the entrance of ...


passo4) usando as colunas de descrição do dataframe a próxima etapa consiste em, para cada uma dessas colunas, gerar um folder de ouput dentro de um outro folder chamado desc_evecs (de description embedding vectors). Esse folder terá o nome do modelo de descrição a partir do qual a descrição sob analise foi gerada. Dentro desse folder, será criado um numpy array .npy com o nome do modelo que gerou os embeddings. Nesse .npy estrão em ordem os vetores de embedding de cada frame do dataframe gerado anteriormente. Isso é feito para cada coluna de descrição e por cada modelo de embedding, tal que um folder desc_evecs/moondream_desc pode possuir multiplos .npy, como msbai-embed-large.npy entre outros.

Observação: problema encontrado ao usar CLIP para fazer embedding das descrições -> esta retornando runtime error. Talvez seja necessário truncar texto para resolver

In [5]:
##############################This section gets the embedding vector of each description model by model#########################################################################
for dir in description_columns:
    description_evec_output_dir = os.path.join(instance_dir, 'desc_evecs', dir)
    os.makedirs(description_evec_output_dir)
    for text_evec_model, model_origin in zip(description_embedding_models['model'],description_embedding_models['model_origin']):
        description_evec_arr_per_model_per_desc_model = np.empty(group_size, dtype=object)
        i = 0
        for description in df[dir]:
            text_evec_scanner = evt.TextEvecScanner(model=text_evec_model, text=description, model_origin=model_origin)
            text_evec = text_evec_scanner.get_evec()
            description_evec_arr_per_model_per_desc_model[i]=text_evec
            i += 1
        output_path_desc_evec = os.path.join(description_evec_output_dir,f'{str(text_evec_model).replace('/','')}.npy')
        np.save(file=output_path_desc_evec, arr=description_evec_arr_per_model_per_desc_model)


passo5) para cada modelo de embedding de imagem, são gerados os vetores de embedding de cada frame e colocados dentro do .npy em ordem como no dataframe. Cada .npy tem o nome do modelo que o gerou e se encontra dentro do folder image_evecs

In [6]:
#############################This section gets the embedding vector for each frame model by model##########################################################################
img_evecs_output_dir = os.path.join(instance_dir, 'img_evecs')
os.makedirs(img_evecs_output_dir)
for image_evec_model, model_origin in zip(image_embedding_models['model'],image_embedding_models['model_origin']):
    image_evec_arr_per_model_per_img_model = np.empty(group_size, dtype=object)
    img_evec_scanner = evt.ImgEVecScanner()
    if model_origin.lower() == "keras":
        img_evec_scanner.add_model(Keras_applications_model=image_evec_model)
    for frame_path in df['frame_path']:
        i = 0
        if model_origin.lower() == "keras":
            res_dict = img_evec_scanner.get_models_evecs(frame_path=frame_path, lin_method='GAP')
        elif model_origin.lower() == "clip":
            res_dict = img_evec_scanner.get_clip_evec(model=image_evec_model, frame_path=frame_path)
        for name, evec in zip(res_dict['model_name'],res_dict['embedding_vector']):
            output_path_img_evec = os.path.join(img_evecs_output_dir,f'{str(name).replace('/','')}.npy')
            image_evec_arr_per_model_per_img_model[i] = evec[0]
            i += 1
    np.save(file=output_path_img_evec, arr=image_evec_arr_per_model_per_img_model)


the new models list is [<Functional name=vgg16, built=True>]
models that will be used have indexes: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 366ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step


segue o exemplo de um dos arrays contendo os vetores de embedding

In [7]:
file_path = 'C:/Documents/Projects_IC/creativision_nyu/UrbanCreativision/evec_scan/instances/test1/img_evecs/vgg16.npy' 
data = np.load(file_path, allow_pickle=True)
print(data)
print("Shape of the array:", data.shape)
print("Data type of the array:", data.dtype)

[array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.08919563e+01, 0.00000000e+00, 0.00000000e+00, 1.23310356e+01,
        0.00000000e+00, 3.64590049e-01, 0.00000000e+00, 0.00000000e+00,
        4.00650978e+00, 0.00000000e+00, 5.57358265e-02, 2.05239177e-01,
        0.00000000e+00, 2.42987704e+00, 0.00000000e+00, 1.88869524e+00,
        9.57921028e+00, 1.29185438e+01, 5.54214239e-01, 0.00000000e+00,
        0.00000000e+00, 1.13503051e+00, 1.59691438e-01, 0.00000000e+00,
        1.65492153e+01, 1.01918347e-01, 1.23666763e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.59477444e+01, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        8.56922436e+00, 0.00000000e+00, 4.18778837e-01, 2.91453886e+00,
        0.00000000e+00, 1.08113192e-01, 8.70395851e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.92704964e+00, 0.00000000e+00,
        6.38081741e+00, 1.54897559e+00, 1.59493402e-01, 5.702902